# Import libraries

In [9]:
import numpy as np 
import torch
import torch.nn.functional as F 
from types import FunctionType
from typing import Tuple

# Different implementations of the convolution function

In [10]:
def get_output_shape(height_input: int, width_input: int, height_conv: int, width_conv: int, padding: int, stride: int) -> Tuple[int, int]:
    height = int((height_input + 2 * padding - (height_conv - 1) - 1) / stride + 1)
    width = int((width_input + 2 * padding - (width_conv - 1) - 1) / stride + 1)
    return height, width

In [11]:
def conv_own(input_data: torch.Tensor, filter_weigts: torch.Tensor, bias: torch.Tensor, stride: int = 1, padding: int = 0) -> torch.Tensor:
    
    channels_input, height_input, width_input = input_data.shape
    numbers_filters, channels_conv, height_conv, width_conv = filter_weigts.shape 

    input_data_padded = F.pad(input=input_data, pad=(padding, padding, padding, padding, 0, 0))
    height_output, width_output = get_output_shape(height_input, width_input, height_conv, width_conv, padding, stride)
    output_data = torch.zeros(size=(numbers_filters, height_output, width_output), dtype=torch.float32) 

    if channels_input != channels_conv:
        raise Exception(f"The number of channels of the source image - {channels_input} and the number of convolution channels - {channels_conv} do not coincide") 
    
    for filter_index in range(numbers_filters):
        for height_output_pixel_index in range(height_output):
            for width_output_pixel_index in range(width_output):
                for height_conv_pixel_index in range(height_conv):
                    for width_conv_pixel_index in range(width_conv):
                        for channel_index in range(channels_input):
                            output_data[filter_index][height_output_pixel_index][width_output_pixel_index] += input_data_padded[channel_index][stride * height_output_pixel_index + height_conv_pixel_index][stride * width_output_pixel_index + width_conv_pixel_index] * filter_weigts[filter_index][channel_index][height_conv_pixel_index][width_conv_pixel_index]

                output_data[filter_index][height_output_pixel_index][width_output_pixel_index] += bias[filter_index]
    
    return output_data


In [12]:
def conv_im2col(input_data: torch.Tensor, filter_weigts: torch.Tensor, bias: torch.Tensor, stride: int = 1, padding: int = 0) -> torch.Tensor:
   
    channels_input, height_input, width_input = input_data.shape
    numbers_filters, channels_conv, height_conv, width_conv = filter_weigts.shape 

    input_data_padded = F.pad(input=input_data, pad=(padding, padding, padding, padding, 0, 0))
    height_output, width_output = get_output_shape(height_input, width_input, height_conv, width_conv, padding, stride)
    output_data = torch.zeros(size=(numbers_filters, height_output, width_output), dtype=torch.float32) 
    input_data_2d = torch.zeros(size=(channels_conv * height_conv * width_conv, height_output * width_output), dtype=torch.float32)
    width_2d = torch.zeros(size=(numbers_filters, channels_conv * height_conv * width_conv), dtype=torch.float32) 

    if channels_input != channels_conv:
        raise Exception(f"The number of channels of the source image - {channels_input} and the number of convolution channels - {channels_conv} do not coincide") 

    for patch_index in range(height_output * width_output): 
        row_index = patch_index % width_output 
        column_index = patch_index // width_output
        input_data_2d[:, patch_index] = input_data_padded[:, stride * row_index: stride * row_index + height_conv, stride * column_index: stride * column_index + width_conv].flatten()
    
    for filter_index in range(numbers_filters): 
        width_2d[filter_index] = filter_weigts[filter_index].flatten()

    output_data = torch.matmul(width_2d, input_data_2d) 
    for filter_index in range(numbers_filters):
        output_data[filter_index] += bias[filter_index] 
    
    output_data = output_data.reshape(numbers_filters, height_output, width_output) 
    output_data = output_data.transpose(1, 2) 
    return output_data 


# Testing

In [13]:
def get_bencmark(in_channels_: int, out_channels_: int, kernel_size_: int, stride_: int, padding_: int, weights: torch.Tensor, bias: torch.Tensor) -> torch.nn.Conv2d:
    conv = torch.nn.Conv2d(in_channels=in_channels_, out_channels=out_channels_, kernel_size=kernel_size_, stride=stride_, padding=padding_)
    conv.weight.data = weights 
    conv.bias.data = bias
    return conv

def get_filter(amount_filters: int, channels: int, kernel_size: int, low_: int = 0, high_: int = 10) -> Tuple[torch.Tensor, torch.Tensor]:
    filter_weights = torch.randint(low=low_, high=high_, size=(amount_filters, channels, kernel_size, kernel_size), dtype=torch.float32)
    bias_ = torch.randint(low=low_, high=high_, size=(amount_filters,), dtype=torch.float32)
    return filter_weights, bias_

def test(input_data: torch.Tensor, func_conv: FunctionType, parameters: dict) -> None:
    test_pass, test_fail = 0, 0
    channels_input, _, _ = input_data.shape

    for amount_filters in parameters['numbers_filters']:
        for kernel_size in parameters['kernel_sizes']:
            for stride in parameters['stride']:
                for padding in parameters['padding']:
                    filter_weights, bias_ =  get_filter(amount_filters, channels_input, kernel_size)
                    bencmark_conv = get_bencmark(channels_input, amount_filters, kernel_size, stride, padding, filter_weights, bias_)
                    
                    bencmark_output = bencmark_conv(input_data)
                    func_conv_output = func_conv(input_data, filter_weights, bias_, stride, padding)

                    if torch.equal(func_conv_output, bencmark_output):
                        test_pass += 1
                        outtext = 'pass'
                    else:
                        test_fail += 1
                        outtext = 'fail'
                        
                print(f'Test {outtext}:\n - amount_filters: {amount_filters}\n - kernel_size: {kernel_size}\n - stride: {stride}\n - padding: {padding}\n')
    
    print(f"Number of tests passed: {test_pass}\nNumber of tests failed: {test_fail}")


In [14]:
# generated input
channels = torch.randint(1, 4, (1,))
shape_ = torch.randint(0, 513, (1,))
print(f'numbers of channels: {channels}\nshape: {shape_}')
input_size = (channels, shape_, shape_)

generated_tensor_input = torch.randint(0, 256, input_size, dtype=torch.float32)

numbers of channels: tensor([2])
shape: tensor([73])


In [15]:
parametrs_1 = {
    'numbers_filters': [1, 5],
    'kernel_sizes': [3, 5],
    'stride': [1, 2],
    'padding': [0, 2]
    }

In [16]:
test(generated_tensor_input, conv_own, parametrs_1) 

Test pass:
 - amount_filters: 1
 - kernel_size: 3
 - stride: 1
 - padding: 2

Test pass:
 - amount_filters: 1
 - kernel_size: 3
 - stride: 2
 - padding: 2

Test pass:
 - amount_filters: 1
 - kernel_size: 5
 - stride: 1
 - padding: 2

Test pass:
 - amount_filters: 1
 - kernel_size: 5
 - stride: 2
 - padding: 2

Test pass:
 - amount_filters: 5
 - kernel_size: 3
 - stride: 1
 - padding: 2

Test pass:
 - amount_filters: 5
 - kernel_size: 3
 - stride: 2
 - padding: 2

Test pass:
 - amount_filters: 5
 - kernel_size: 5
 - stride: 1
 - padding: 2

Test pass:
 - amount_filters: 5
 - kernel_size: 5
 - stride: 2
 - padding: 2

Number of tests passed: 16
Number of tests failed: 0


In [17]:
parametrs_2 = {
    'numbers_filters': [1, 5, 20],
    'kernel_sizes': [3, 5, 7, 16, 32],
    'stride': [1, 2, 3],
    'padding': [0, 1, 2, 4]
    }

test(generated_tensor_input, conv_im2col, parametrs_2)

Test pass:
 - amount_filters: 1
 - kernel_size: 3
 - stride: 1
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 3
 - stride: 2
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 3
 - stride: 3
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 5
 - stride: 1
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 5
 - stride: 2
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 5
 - stride: 3
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 7
 - stride: 1
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 7
 - stride: 2
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 7
 - stride: 3
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 16
 - stride: 1
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 16
 - stride: 2
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 16
 - stride: 3
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 32
 - stride: